In [1]:
%run ~/relmapping/annot/notebooks/__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/beegfs/scratch_copy/ahringer/jj374/lab/relmapping


In [2]:
fp_regl = 'annot/Fig2D2_regulatory_annotation_Apr27/S2_regulatory_annotation_Apr27.tsv'
df_regl = pd.read_csv(fp_regl, sep='\t')

df_regl = df_regl[['chrom', 'start', 'end', 'promoter_locus_id_fwd', 'promoter_locus_id_rev', 'promoter_gene_id_fwd', 'promoter_gene_id_rev', 'annot_fwd', 'annot_rev', 'annot']]
print('%d regions loaded' % (len(df_regl),))

42245 regions loaded


In [6]:
def df_mid(df_inp):
    df_out = df_inp.copy()
    df_out['start'] = df_out[['start', 'end']].mean(axis=1).astype(int).copy()
    df_out['end'] = df_out['start'] + 1
    return df_out.reset_index(drop=True)

def df_closest_by_mid(df_a_, df_b_, b_prefix, **kwargs):
    df_a = df_mid(df_a_)
    df_b = df_mid(df_b_)
    fn_ = BedTool.from_dataframe(df_a).closest(BedTool.from_dataframe(df_b).fn, **kwargs).fn
    names_ = list(itertools.chain(df_a.columns.values,
        ['%s_%s' % (b_prefix, col) for col in df_b.columns.values],
        ['%s_distance' % (b_prefix)]
    ))
    df_ = pd.read_csv(fn_, names=names_, sep='\t')
    return pd.concat([df_a_, df_[names_[len(df_a.columns):]]], axis=1)


#fp_inp,fp_out,fp_bed = 'annot_cfp1/cfp1_summit_peaks_300bp.bed', 'annot_cfp1/cfp1_summit_peaks_300bp_annotated.tsv', 'annot_cfp1/cfp1_summit_peaks_300bp_annotated.bed'
#fp_inp,fp_out,fp_bed = 'annot_cfp1/HDA1_summit_peaks_300bp.bed', 'annot_cfp1/HDA1_summit_peaks_300bp_annotated.tsv', 'annot_cfp1/HDA1_summit_peaks_300bp_annotated.bed'
#fp_inp,fp_out,fp_bed = 'annot_cfp1/SIN3_all_summit_peaks_300bp.bed', 'annot_cfp1/SIN3_all_summit_peaks_300bp_annotated.tsv', 'annot_cfp1/SIN3_all_summit_peaks_300bp_annotated.bed'

df_inp = pd.read_csv(fp_inp, sep='\t', names=yp.NAMES_BED6).sort_values(yp.NAMES_BED3).reset_index(drop=True)
#(df_inp['end'] - df_inp['start'] == 300).all() #=> all peaks are 300bp

# find closest site
df_out = df_closest_by_mid(df_inp, df_regl, b_prefix='regl', d=True, t='first')
df_out.tail()

# clear hits further than 150bp
m = abs(df_out.regl_distance) > 150
df_out.loc[m, df_out.columns[6:]] = float('nan')

# write .tsv, .bed-file for browsing
df_out.to_csv(fp_out, sep='\t')
write_gffbed(fp_bed,
    chrom = df_out['chrom'],
    start = df_out['start'],
    end = df_out['end'],
    attr = df_out[df_out.columns[6:]],
)